# Set configuration

You need to login to a server and fill the following variables:

In [ ]:
from IPython.core.display import display
import ipywidgets as widgets

session_widget = widgets.Text(placeholder='PHPSESSID', description='Session ID')
server_widget = widgets.Text(value='s1.railnation.de', description='Server URL')

display(session_widget, server_widget)

In [ ]:
session_id = session_widget.value
server_url = server_widget.value

%load_ext autoreload
%autoreload 2

In [ ]:
# get user ID
from server import ServerCaller

api = ServerCaller(server_url, session_id)
server = api.call('ServerInfoInterface', 'getInfo', None)
user_id = server['Infos']['activeUser']
print(f'Logged in to {server["Body"]["worldName"]}')

In [ ]:
# association analysis
assoc_info = api.call('CorporationInterface', 'getOtherMemberBuildings',
                      ["00000000-0000-0000-0000-000000000000"])
# 7: Hotel, 8: Restaurant, 9: Mall
print(f"Info for {len(assoc_info['Body'])} members")

In [ ]:
buildings = {
"7": api.properties('building_hotel', version='8984d44e15dc4d3481b18a42aa1969e6'),
"8": api.properties('building_restaurant', version='8984d44e15dc4d3481b18a42aa1969e6'),
"9": api.properties('building_mall', version='8984d44e15dc4d3481b18a42aa1969e6'),
}

In [ ]:
from statistics import mean
from collections import defaultdict

all_balances = [int(value['moneyAmount']) for _, value in assoc_info['Body'].items()]
print(f"Current mean balance of all members: {mean(all_balances):,.0f}")

member_buildings = defaultdict(lambda: dict())


for user_id, member in assoc_info['Body'].items():
    for key, value in member.items():
        if key in buildings:
            building = buildings[key]
            member_buildings[user_id][building['name']] = building['levels'][value['level']]['effect']

all_restaurants = [float(value['Restaurant']['FARM']['MONEY']) * (float(member_buildings[user_id]['Hotel']['INCOME_BONUS']['MONEY']) / 100 + 1)
                   for user_id, value in member_buildings.items()]
all_malls = [float(value['Mall']['FARM']['MONEY']) * (float(member_buildings[user_id]['Hotel']['INCOME_BONUS']['MONEY']) / 100 + 1)
             for user_id, value in member_buildings.items()]
all_hotels = [int(value['Hotel']['FARM']['PRESTIGE']) for _, value in member_buildings.items()]


# daily income
durations = {
    'Restaurant': 90,
    'Mall': 360,
    'Hotel': 180
}

restaurant_income = mean(all_restaurants) * (24 * 60 / durations['Restaurant'])
mall_income = mean(all_malls) * (24 * 60 / durations['Mall'])
total_income = restaurant_income + mall_income

hotel_income = mean(all_hotels) * (24 * 60 / durations['Hotel'])
print('- Regular -')
print(f"Mean income per member per day: {total_income:,.0f}")
print(f"Mean prestige per member per day: {hotel_income:,.0f}")

# Leonardo
durations_worker = {k: v * 0.6 for k, v in durations.items()}

restaurant_income = mean(all_restaurants) * (24 * 60 / durations_worker['Restaurant'])
mall_income = mean(all_malls) * (24 * 60 / durations_worker['Mall'])
total_income = restaurant_income + mall_income

hotel_income = mean(all_hotels) * (24 * 60 / durations_worker['Hotel'])

print('- With worker -')
print(f"Mean income per member per day: {total_income:,.0f}")
print(f"Mean prestige per member per day: {hotel_income:,.0f}")
